In [4]:
!pip install librosa transformers torch soundfile audioread ipywidgets --upgrade
!pip install jupyterlab_widgets imageio[ffmpeg] pydub

In [5]:
import os
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from pydub import AudioSegment
import torch

In [6]:
# Paths to datasets
btp_path = "notebooks/BTP.csv"
agc_path = "notebooks/AGC.csv"
weather_stats_path = "notebooks/weather_stats.csv"

# Load datasets
btp_data = pd.read_csv(btp_path)
agc_data = pd.read_csv(agc_path)
weather_stats_data = pd.read_csv(weather_stats_path)

# Standardize date-time columns
btp_data['valid'] = pd.to_datetime(btp_data['valid'], errors='coerce')
agc_data['valid'] = pd.to_datetime(agc_data['valid'], errors='coerce')
weather_stats_data['timestamp'] = pd.to_datetime(weather_stats_data.get('timestamp'), errors='coerce')

print("Datasets loaded successfully.")

FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/BTP.csv'